<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-25 13:20:56
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.35 C
-------------------
Today PnL: -2.02 L (-1.48%)
Current PnL: -16.59 L (-11.56%)
CY Booked + Current PnL: -8.51 L (-5.93%)
-------------------
Total profit:  2.91 L
Total loss:  -19.51 L
-------------------
Total Booked + Current PnL: 18.47 L (15.84%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.0%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 74.81 L (55.59%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 9.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.68,H-SC,11.25,87760.0,-7556.0,7591.0,-0.28,-7.93,8.65,0.04,161.0,-1.00,0.65,73.10,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.44,110736.0,6424.0,11107.0,-0.50,6.16,10.03,16.81,9.0,0.58,0.82,23.19,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-18.07,H-LC,6.72,111578.0,8012.0,12698.0,1.02,7.74,11.38,20.00,14.0,0.63,0.83,22.90,X40N,BTT,HEALTHCARE
70,SBIN,760.30,863.00,-11.51,M-LC,4.71,214412.0,12932.0,14280.0,-0.81,6.42,6.66,13.51,58.0,0.91,1.59,19.82,XY25,NTT,BANKS
17,BRITANNIA,5190.70,6446.05,12.68,H-LC,9.92,101043.0,7610.0,14985.0,-1.05,8.15,14.83,24.18,32.0,0.51,0.75,22.69,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.0,87.81,M-MC,14.1,224979.0,17.0,167632.0,-4.05,0.01,74.51,74.53,193.0,0.0,1.67,30.62,XY24,BTT,STEEL


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-21.48,H-LC,0.80,206281.0,1781.0,39627.0,1.20,0.87,19.21,20.25,11.0,0.04,1.53,9.68,X40N,BTT,PHARMA
10,BAJAJHFL,122.26,152.00,-7.54,H-LC,2.24,191962.0,-8544.0,57320.0,-2.18,-4.26,29.86,24.33,23.0,-0.15,1.43,10.70,X40N,BTT,FINANCE
56,LTIM,5564.16,7230.20,-3.63,H-LC,2.56,185238.0,-15072.0,75058.0,-1.16,-7.52,40.52,29.94,28.0,-0.20,1.38,27.97,X200,ATH,IT
40,HINDUNILVR,2413.81,2723.00,-13.63,H-LC,2.65,270659.0,312.0,34320.0,-0.87,0.12,12.68,12.81,16.0,0.01,2.01,13.19,XY25,NTT,FMCG
41,HINDZINC,471.44,730.22,17.72,H-LC,2.98,192379.0,-12697.0,125258.0,-0.86,-6.19,65.11,54.89,3.0,-0.10,1.43,16.41,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TRIDENT,37.35,48.00,28.76,M-SC,4.85,80892.0,-11362.0,37663.0,3.87,-12.32,46.56,28.51,239.0,-0.30,0.60,38.42,XR,NTT,TEXTILES
22,CIPLA,1492.70,1795.00,-21.48,H-LC,0.80,206281.0,1781.0,39627.0,1.20,0.87,19.21,20.25,11.0,0.04,1.53,9.68,X40N,BTT,PHARMA
4,APOLLOHOSP,6904.43,8285.00,-18.07,H-LC,6.72,111578.0,8012.0,12698.0,1.02,7.74,11.38,20.00,14.0,0.63,0.83,22.90,X40N,BTT,HEALTHCARE
16,BLUSPRING,226.45,94.95,NaN,NaN,4.76,20720.0,-28873.0,75.0,0.96,-58.22,0.36,-58.07,201.0,-384.97,0.15,38.95,XY24,ATH,MISC
58,MEDANTA,1087.93,1486.00,20.66,H-SC,9.85,154616.0,28416.0,17765.0,0.50,22.52,11.49,36.59,151.0,1.60,1.15,37.85,XY24,NTT,HEALTHCARE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,NESTLEIND,2268.60,2755.0,-14.37,H-LC,5.74,271686.0,6260.0,50642.0,-5.32,2.36,18.64,21.44,19.0,0.12,2.02,9.16,XY25,NTT,FMCG
86,UJJIVANSFB,52.77,53.0,35.36,M-SC,23.15,117666.0,-24813.0,25439.0,-4.66,-17.42,21.62,0.44,249.0,-0.98,0.87,38.88,OX40N,NTT,BANKS
67,SAIL,130.64,228.0,87.81,M-MC,14.10,224979.0,17.0,167632.0,-4.05,0.01,74.51,74.53,193.0,0.00,1.67,30.62,XY24,BTT,STEEL
57,MASFIN,326.60,399.5,-15.54,H-SC,17.02,96045.0,-1935.0,23810.0,-3.79,-1.97,24.79,22.32,148.0,-0.08,0.71,39.30,XR,ATH,FINANCE
77,SURYODAY,189.78,240.0,49.12,H-SC,4.36,105307.0,-44619.0,84298.0,-3.71,-29.76,80.05,26.46,146.0,-0.53,0.78,34.91,XR,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.68,H-SC,11.25,87760.0,-7556.0,7591.0,-0.28,-7.93,8.65,0.04,161.0,-1.00,0.65,73.10,OX40N,NTT,MISC
78,SYMPHONY,1306.42,1306.0,-31.27,M-SC,10.12,146288.0,-24853.0,24796.0,-1.10,-14.52,16.95,-0.03,194.0,-1.00,1.09,5.86,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-17.98,H-MC,1.39,86163.0,-16489.0,16483.0,-1.15,-16.06,19.13,-0.00,101.0,-1.00,0.64,30.98,OX40N,NTT,IT
21,CERA,7989.07,8422.0,-16.71,H-SC,3.17,119196.0,-24607.0,32397.0,-1.28,-17.11,27.18,5.42,154.0,-0.76,0.89,31.54,OX40N,NTT,CERAMICS
86,UJJIVANSFB,52.77,53.0,35.36,M-SC,23.15,117666.0,-24813.0,25439.0,-4.66,-17.42,21.62,0.44,249.0,-0.98,0.87,38.88,OX40N,NTT,BANKS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VBL,492.64,672.28,-14.67,H-LC,13.10,259128.0,-5912.0,102563.0,-0.89,-2.23,39.58,36.46,1.0,-0.06,1.93,10.67,X40N,ATH,FMCG
32,ENRIN,1377.95,3176.70,NaN,NaN,2.47,74360.0,39911.0,5056.0,-1.03,115.86,6.80,130.54,2.0,7.89,0.55,13.71,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,17.72,H-LC,2.98,192379.0,-12697.0,125258.0,-0.86,-6.19,65.11,54.89,3.0,-0.10,1.43,16.41,X5K,ATH,METALS
82,TCS,3794.03,4998.00,-24.81,H-LC,12.01,245435.0,-50499.0,144414.0,-0.04,-17.06,58.84,31.73,4.0,-0.35,1.83,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-15.29,H-LC,6.96,272187.0,10586.0,135032.0,-2.05,4.05,49.61,55.67,6.0,0.08,2.02,10.40,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,1.77,55306.0,14051.0,3385.0,-1.53,34.06,6.12,42.26,197.0,4.15,0.41,30.21,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,20.66,H-SC,9.85,154616.0,28416.0,17765.0,0.50,22.52,11.49,36.59,151.0,1.60,1.15,37.85,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.47,74360.0,39911.0,5056.0,-1.03,115.86,6.80,130.54,2.0,7.89,0.55,13.71,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,87.81,M-MC,14.10,224979.0,17.0,167632.0,-4.05,0.01,74.51,74.53,193.0,0.00,1.67,30.62,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,28.70,H-SC,12.80,131771.0,6474.0,52603.0,-2.16,5.17,39.92,47.15,144.0,0.12,0.98,64.85,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,20.66,H-SC,9.85,154616.0,28416.0,17765.0,0.50,22.52,11.49,36.59,151.0,1.60,1.15,37.85,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,0.98,H-SC,1.58,113464.0,11460.0,32882.0,-3.44,11.23,28.98,43.48,121.0,0.35,0.84,29.90,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-6.04,H-SC,10.63,176280.0,11610.0,56357.0,-0.09,7.05,31.97,41.28,119.0,0.21,1.31,15.60,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.47,74360.0,39911.0,5056.0,-1.03,115.86,6.80,130.54,2.0,7.89,0.55,13.71,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-37.67,M-SC,9.20,104775.0,13276.0,65474.0,-0.98,14.51,62.49,86.07,207.0,0.20,0.78,51.57,XR,NTT,DURABLES
53,KPIGREEN,497.21,731.64,28.70,H-SC,12.80,131771.0,6474.0,52603.0,-2.16,5.17,39.92,47.15,144.0,0.12,0.98,64.85,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-47.89,H-SC,12.83,135855.0,12519.0,121251.0,-2.20,10.15,89.25,108.45,118.0,0.10,1.01,34.77,AR,ATH,MISC
95,WIPRO,243.46,420.00,-6.95,M-LC,5.62,159588.0,8643.0,100812.0,-1.70,5.73,63.17,72.51,55.0,0.09,1.19,9.93,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,4.76,20720.0,-28873.0,75.0,0.96,-58.22,0.36,-58.07,201.0,-384.97,0.15,38.95,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,1.77,55306.0,14051.0,3385.0,-1.53,34.06,6.12,42.26,197.0,4.15,0.41,30.21,XY24,ATH,IT
74,SIS,477.00,477.00,2315.03,M-SC,5.68,64974.0,-16116.0,16114.0,-0.18,-19.87,24.80,0.00,236.0,-1.00,0.48,31.68,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.27,69408.0,-44141.0,84192.0,-1.87,-38.87,121.30,35.27,266.0,-0.52,0.52,69.88,XR,NTT,HOTELS
84,TRIDENT,37.35,48.00,28.76,M-SC,4.85,80892.0,-11362.0,37663.0,3.87,-12.32,46.56,28.51,239.0,-0.30,0.60,38.42,XR,NTT,TEXTILES


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,4.76,20720.0,-28873.0,75.0,0.96,-58.22,0.36,-58.07,201.0,-384.97,0.15,38.95,XY24,ATH,MISC
33,GREENPANEL,375.16,537.00,245.61,M-SC,10.39,154208.0,-28870.0,107853.0,-0.47,-15.77,69.94,43.14,223.0,-0.27,1.15,45.05,XY24,NTT,MISC
29,DIXON,14367.00,18940.15,-27.21,H-MC,3.91,134000.0,19064.0,17527.0,-0.11,16.59,13.08,31.83,83.0,1.09,1.00,53.58,X40N,ATH,IT
58,MEDANTA,1087.93,1486.00,20.66,H-SC,9.85,154616.0,28416.0,17765.0,0.50,22.52,11.49,36.59,151.0,1.60,1.15,37.85,XY24,NTT,HEALTHCARE
28,DIGITIDE,188.38,267.99,NaN,NaN,1.77,55306.0,14051.0,3385.0,-1.53,34.06,6.12,42.26,197.0,4.15,0.41,30.21,XY24,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,41.13
2,50,69.99


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.84
LC    35.01
MC    22.05
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.87
X40N     14.52
X40      12.48
XY25     10.75
XR        9.23
OX40N     8.36
AR        8.26
X5K       2.25
MH        1.75
X200      1.38
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.17
H-SC    25.42
H-MC    19.02
M-SC    14.99
M-LC     5.79
M-MC     2.68
L-SC     1.43
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.06,-3.54,38.18
IT,11.87,-13.92,70.05
FINANCE,8.36,-15.94,58.01
BANKS,7.58,-11.81,57.43
MISC,6.05,-18.09,73.24
PAINTS,5.59,-16.15,39.04
ELECTRICAL,5.42,-0.99,48.85
HEALTHCARE,4.58,1.79,24.30
AUTO,4.37,-14.85,64.81


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2700969.0,25
XR,1053128.0,13
AR,1034127.0,10
X40,773950.0,10
X40N,598875.0,12
XY25,491391.0,8
OX40N,348323.0,11
SR,187712.0,2
X5K,136365.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2402684.0,26
M-SC,1615647.0,20
H-LC,1392860.0,21
H-MC,1149165.0,15
M-MC,317221.0,2
M-LC,282394.0,4
L-SC,208511.0,3
L-MC,55593.0,1
L-LC,48132.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,939429.0,7
M-SC,XY24,776521.0,7
H-SC,AR,513394.0,3
H-LC,X40,501599.0,6
H-MC,XY24,444321.0,4
H-SC,XR,350772.0,4
M-MC,XY24,317221.0,2
H-LC,X40N,316104.0,6
M-SC,XR,279355.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
